In [5]:
import torch

from wearsed.dataset.WearSEDDataset_vanilla import WearSEDDataset

In [2]:
def get_random_sequence(signals, labels, max_time, seq_length):
    start = torch.randint(0, max_time, (1,))
    end = start + seq_length
    return signals[:, start:end], labels[start:end]

def get_batch(signals, labels, batch_size, seq_length):
    max_time = signals.shape[1] - seq_length

    tries = 0
    batch_signals = []
    batch_labels = []
    for i in range(batch_size):
        signals_seq, labels_seq = get_random_sequence(signals, labels, max_time, seq_length)
        if i < 0.8*batch_size or tries < 20:  # If there are not enough sequences with events and we haven't tried long enough
            if labels_seq.sum() == 0:         # ..check if this is a sequence without events and if so, roll again
                signals_seq, labels_seq = get_random_sequence(signals, labels, max_time, seq_length)  # TODO could still be negative sample
                batch_signals.append(signals_seq)
                batch_labels.append(labels_seq)
                tries += 1
                continue
        batch_signals.append(signals_seq)
        batch_labels.append(labels_seq)
    return torch.stack(batch_signals), torch.stack(batch_labels)

In [ ]:
dataset = WearSEDDataset(mesaid_path='../wearsed/dataset/data_ids/', signals_to_read=['SpO2', 'Pleth'], preprocess=True)

In [14]:
for i in range(20):
    signals, labels = dataset[i]
    _, y = get_batch(signals, labels, batch_size=64, seq_length=600)
    print((y.sum(dim=1)>0).sum().item(), '/', y.shape[0])

42 / 64
57 / 64
63 / 64
37 / 64
37 / 64
41 / 64
62 / 64
54 / 64
49 / 64
48 / 64
25 / 64
54 / 64
52 / 64
56 / 64
34 / 64
56 / 64
57 / 64
44 / 64
37 / 64
59 / 64
